# CSMODEL S11 | Project Phase 1
This notebook is the work of Group 4, consisting of the following members:

* CARNEY, JOHN PAUL COMPANIA
* GUERRRERO, MIGUEL ALFONSO DAVID
* REINANTE, CHRISTIAN VICTOR GO
* SALVADOR, JARYLL FRANCIS PENA

## Dataset Description
This project makes use of the [Online Gaming Anxiety Data Set](https://www.kaggle.com/datasets/divyansh22/online-gaming-anxiety-data). It contains responses gathered from a worldwide survey of gamers. Included in this survey are psychological assessments for anxiety, social phobia, and life satisfaction. It also gathered demographic and gaming-related information. Marian Sauter and Dejan Draschkow originally compiled the data.


## Importing Libraries
Before proceeding, we will import the necessary libraries which we will use to provide a general overview of the dataset.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Importing Libraries
We then load the dataset as follows:

In [ ]:
gamingAnxiety_df = pd.read_csv("GamingStudy_data.csv")
gamingAnxiety_df.head()

## Process and Implications of Data Collection
The data was gathered by means of a survey that was distributed to gamers globally. The survey had a range of inquiries commonly employed by psychologists to assess levels of anxiety, social phobia, and life satisfaction. Standardized psychological assessment instruments, including the General Anxiety Disorder Assessment (GAD), Satisfaction with Life Scale (SWL), and Social Phobia Inventory (SPIN) questionnaires, and inquiries regarding gaming habits and general demographics were included in the survey. 

Though not explicitly mentioned, it is extremely likely that this survey was conducted online, given that online surveys are commonly used when reaching a worldwide audience, especially gamers. The dataset description also includes *Reddit* as an example for the **Reference** variable, indicating the website was used as an avenue to conduct the survey as well. Assuming the data was collected as such, this presents several implications:

- **Sample Composition**: Because the data was collected through an online survey, it may over-represent individuals active in online gaming communities or gamers who primarily play online multiplayer games. As a result, those who do not regularly use the internet, are inactive in online gaming communities, or those who play single-player games exclusively may be underrepresented.

- **Voluntary Response Bias**: The data relies on self-reported responses, which can be subject to biases such as inaccurate self-assessment by the respondent or social desirability bias. Respondents with stronger views also may have been more likely to participate in the first place because of this.

**Each row** represents a single survey response from a gamer, and **each column** represents a variable collected in the survey. The dataset contains **13464 observations** in total, and there are **55 variables** in the dataset. We can verify this, and also check each individual variable using the info() method:

In [ ]:
gamingAnxiety_df.info()

#### Demographic Information

- **S. No.:** Serial Number.  
- **Timestamp:** Time at which the participant took the questionnaire after it being launched.  
- **Gender:** Self-identified gender of the gamer taking the questionnaire.  
- **Age:** Self-reported age of the gamer taking the questionnaire.  
- **Work:** Work status of the gamer.  
- **Degree:** Highest degree attained.  
- **Birthplace:** Birthplace.  
- **Residence:** Place where the gamer currently resides.  
- **Residence_ISO3:** Current residence in ISO3 format.  
- **Birthplace_ISO3:** Birthplace in ISO3 format.
- **Accept:** Accept terms and conditions (not necessary for any analysis).  

#### Psychological Assessment

- **GAD1 to GAD7:** Responses to GAD questions 1 to 7.  
- **GADE:** Effect of gaming on work.  
- **SWL1 to SWL5:** Responses to SWL questions 1 to 5.  
- **SPIN1 to SPIN17:** Responses to SPIN questions 1 to 17.  
- **Narcissism:** Interest scale in the game (1-5).  
- **GAD_T:** GAD Total Score.  
- **SWL_T:** SWL Total Score.  
- **SPIN_T:** SPIN Total Score.  

#### Gaming Habits

- **Game:** Name of the game they play.  
- **Platform:** Mode of game playing (PC, Console, Mobile, etc.).  
- **Hours:** Number of hours in a week devoted to playing.  
- **earnings:** Earnings from the game (if any).  
- **whyplay:** Reason to play the game.  
- **League:** Respondent's current ingame rank.  
- **highestleague:** Highest rank attained.  
- **streams:** Number of online streaming sessions.


## Data Cleaning 
Next, we prepare our dataset for modeling and analysis. 

#### Pinpoint and Remove Irrelevant Variables
We start by removing the following variables:
- **League:** This column has inconsistent formatting and its value is not utilized in the study.
- **highestleague:** This column is entirely consisting of null values and will not be used.
- **Accept:** This column is not neccessary for analysis.
- **earnings:** This column is relevant to the study.
- **streams:** This column is not relevant to the study.
- **Residence:** We will be using Residence_ISO3 instead, as it is formatted more consistently.
- **Birthplace:** We will be using Birthplace_ISO3 instead, as it is formatted more consistently.

In [ ]:
gamingAnxiety_df = gamingAnxiety_df.drop(columns=['League', 'highestleague', 'accept', 'earnings', 'streams', 'Residence', 'Birthplace'])

#### Handling Null Values 
This section will place focus on the Psychological Assessment variables as well as the gaming habits. To start, we drop variables that are irrelevant to our study. We will then start looking for variables with null values. We do this by iterating over each column and checking how many null-valued cells each of these may have.

In [ ]:
nullVariables = gamingAnxiety_df.columns[gamingAnxiety_df.isnull().any()].tolist()
gamingAnxiety_df[nullVariables].isnull().sum()

Most variables here have a relatively low amount off null values (Less than 5%). Although we could choose to drop this data given how few they are, we will choose to perform imputation to preserve our sample size and maintain the variability of our dataset. Furthermore, if the missing cells are scattered (i.e. many rows only have one or two cells missing), then we may end up dropping a deceptively high amount of rows rather than just a few hundred. At worst, we may end up dropping a number of rows equal to the sum of the number of null values we have. 

We start by doing mean imputation. Of course, we will only be doing this for cells that are supposed to have a numerical value.

In [ ]:
columns_to_impute = ['Hours', 'Narcissism', 'SPIN_T',
                     'SPIN1', 'SPIN2', 'SPIN3', 'SPIN4', 'SPIN5', 
                     'SPIN6', 'SPIN7', 'SPIN8', 'SPIN9', 'SPIN10', 
                     'SPIN11', 'SPIN12', 'SPIN13', 'SPIN14', 'SPIN15', 
                     'SPIN16', 'SPIN17']

for column in columns_to_impute:
    gamingAnxiety_df[column] = gamingAnxiety_df[column].fillna(gamingAnxiety_df[column].mean())


Let's verify that we've successfully performed the imputation:

In [ ]:
gamingAnxiety_df[nullVariables].isnull().sum()

We cannot use mean imputation for our categorical variables. We would also rather not drop them, for the same reason we do not want to drop our numerical variables. One method of imputation compatible with categorical values we can use is mode imputation. We mode imputation below:

In [ ]:
columns_to_impute = ['GADE', 'Work', 'Degree', 'Reference',
                     'Residence_ISO3', 'Birthplace_ISO3']

for column in columns_to_impute:
    mode_value = gamingAnxiety_df[column].mode().iloc[0]
    gamingAnxiety_df[column] = gamingAnxiety_df[column].fillna(mode_value)

And again verify that we've successfully imputed the categorical variables we've targetted:

In [ ]:
gamingAnxiety_df[nullVariables].isnull().sum()

## Exploratory Data Analysis
Exploratory Data Analysis (EDA) is a crucial step in understanding the characteristics and underlying patterns in our dataset. In this study, we conducted a comprehensive EDA to explore the psychological measures of anxiety, life satisfaction, and social phobia among gamers worldwide. 

In [ ]:
gamingAnxiety_df.head()

## 1st : Distribution of Key Psychological Measures
What is the distribution of anxiety, life satisfaction, and social phobia scores among gamers?

To answer this we take have to take a comprehensive look into the psychological state of the respondents and, to analyze the distribution of three key psychological measures: anxiety scores (GAD_T), life satisfaction scores (SWL_T), and social phobia scores (SPIN_T). We will use histograms to visualize the frequency distributions of these measures, as they provide an intuitive way to see how scores are spread across different ranges and to identify common patterns and abnormalities.

In [ ]:
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
sns.histplot(gamingAnxiety_df['GAD_T'], kde=True, bins=30)
plt.title('Distribution of Anxiety Scores (GAD_T)')
plt.xlabel('Anxiety Score')
plt.ylabel('No. of Respondents')

plt.subplot(1, 3, 2)
sns.histplot(gamingAnxiety_df['SWL_T'], kde=True, bins=30)
plt.title('Distribution of Life Satisfaction Scores (SWL_T)')
plt.xlabel('Life Satisfaction Score')
plt.ylabel('No. of Respondents')

plt.subplot(1, 3, 3)
sns.histplot(gamingAnxiety_df['SPIN_T'], kde=True, bins=30)
plt.title('Distribution of Social Phobia Scores (SPIN_T)')
plt.xlabel('Social Phobia Score')
plt.ylabel('No. of Respondents')

plt.tight_layout()
plt.show()


## Explanation : 
The data given shows that the anxiety frequency distribution has most respondents have lower anxiety score. The distribution of anxiety scores shows that most of the respondents medium to high life satisfaction scores. Lastly, the social phobia scores shows that most respondents have a lower phobia scores, with fewer respondents reporting higher levels of social phobia.

Let's explain it more. For the anxiety frequency distribution or the GAD-T, it shows a right-skewed distribution that indicates most repsondents have lower anxiety scores, The distribution helps us understand that anxiety levels are generally low among the respondents, with some outliers experiencing higher levels of anxiety. We can apply this data by analyzing this distrubtion, we can identify the proportion of respondents experiencing varying degrees of anxiety, which can help in data gathering and usage. For the Life Satisfaction scores, or the SWL_T, the histogram displays the distribution with a slightly left-skewed shape suggesting most respondents have moderate to high life satisfaction, or they're currently happy with their lives right now. This shows that the overall sample of respondents have a overall well-being. Now for the Social Phobia Scores or SPIN_T, the histogram shows the distribution is a right-skewed shape indicating that most repsondents have a lower social phobia score. Showing that social anxiety is not a significant issue for most of the respondents, with only a small minority experiencing high levels of social phobia. 

Below is the description or values assigned to the GAD-T, SWL-T and the SPIN-T with a summary of each of their Mean, Medians, and the Standard Deviation.

## Conclusion:
According to the data given, the distribution when it comes to the GAD-T shows that most of the respondents have lower anxiety scores, we can assume that majority of the respondents do not have any problems with anxiety. Same with the others, as the graph shows that the SWL-T although left-skewed shows that majority of the respondents are quite happy with their lives. SPIN-T graphs shows that majority of the respondents do not suffer from social phobia and only a small minority do. 

##  Gaming Hours
Here, we seek to answer the question "Do gaming hours per week correlate with anxiety, life satisfaction, and social phobia scores?"
 similarly plot the data as follows:

In [ ]:
gamingAnxiety_df = gamingAnxiety_df[gamingAnxiety_df['Hours'] <= 100]
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
sns.scatterplot(x='Hours', y='GAD_T', data=gamingAnxiety_df)
plt.title('Gaming Hours per Week vs Anxiety Scores')
plt.xlabel('Gaming Hours per Week')
plt.ylabel('Anxiety Score')

plt.subplot(1, 3, 2)
sns.scatterplot(x='Hours', y='SWL_T', data=gamingAnxiety_df)
plt.title('Gaming Hours per Week vs Life Satisfaction Scores')
plt.xlabel('Gaming Hours per Week')
plt.ylabel('Life Satisfaction Score')

plt.subplot(1, 3, 3)
sns.scatterplot(x='Hours', y='SPIN_T', data=gamingAnxiety_df)
plt.title('Gaming Hours per Week vs Social Phobia Scores')
plt.xlabel('Gaming Hours per Week')
plt.ylabel('Social Phobia Score')

plt.tight_layout()
plt.show()

## Explanation
Scatterplots are best used when trying to visualize a relationship between two continous variables. Here we attempt to visualize any potential relationship between GAD_T, SPIN_T, and SWL_T.

From visuals alone, there is no obvious trend between gaming hours per week and these variables. One thing of note, however, is that the dots tend to be concentrated towards the left side of the graph. This skewness does indicate that the dataset represents more observations with lower gaming hours per week. Besides this, there is little more that can be gathered from looking at the graph alone.

## Conclusion:
Attempting to visualize potential relationships between gaming hours and anxiety, social phobia, and life satisfaction shows no clear trend. However, the concentration of dots on the left suggests lower gaming hours overall.

## Demographic breakdown (Age, Gender, Nationality) of Gamers
What is the demographic breakdown (age, gender, nationality) of gamers in the survey?
For this we can barplot the entire dataset to see the density of the respondents and where they live.

In [ ]:
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
sns.histplot(gamingAnxiety_df['Age'], kde=True, bins=30)
plt.title('Distribution of Age')
plt.xlabel('Age')
plt.ylabel('Frequency')

plt.subplot(1, 3, 2)
sns.countplot(x='Gender', data=gamingAnxiety_df)
plt.title('Gender Distribution')
plt.xlabel('Gender')
plt.ylabel('Count')

plt.subplot(1, 3, 3)
top_nationalities = gamingAnxiety_df['Birthplace'].value_counts().head(10)
sns.barplot(x=top_nationalities.index, y=top_nationalities.values)
plt.title('Top 10 Nationalities')
plt.xlabel('Nationality')
plt.ylabel('Count')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

## Explanation
With the first graph being the age distribution we can see that the majority of the respondents are in their mid-20s, with fewer respondents on the outliers, being younger and older age extremes. The second graph shows the counts of male, female and other respondents. It is apparent that the respondents of this dataset is majorly males, with smaller proportions of female and other genders. The last graph being the top 10 nationalities of respondents. These are namely USA, Germany, UK, Canada, Netherlands, France, Sweden, Poland, Brazil and Portugal.

Let's dive in deeper in the graphs. The first graph shows that the peak respondent age is mid-20s, this shows that the age group is highly represented in this sampling, then we have more extremes reaching up to 60, showing that gaming is not a phase but a passion for some people. The second graph shows that majority of the respondents are male, we can use this for mental health data research and in indentifying target demographics. The last graph is again, the Top 10 Nationalities, showing USA as first, then Germany. The data also shows that 80% of the Top 10 countries are in Europe.


## Conclusion
The vast majoriy of gamers from the dataset are below the age of 20, Male, and from the USA.

## Relationship Between Preferred Gaming Device, Game Genre, and Psychological Measures

In [ ]:
# Preferred gaming device and psychological measures
plt.figure(figsize=(15, 10))

# Device vs Anxiety
plt.subplot(3, 1, 1)
sns.boxplot(x='Platform', y='GAD_T', data=gamingAnxiety_df)
plt.title('Preferred Gaming Device vs Anxiety Scores')
plt.xlabel('Gaming Device')
plt.ylabel('Anxiety Score')

# Device vs Life Satisfaction
plt.subplot(3, 1, 2)
sns.boxplot(x='Platform', y='SWL_T', data=gamingAnxiety_df)
plt.title('Preferred Gaming Device vs Life Satisfaction Scores')
plt.xlabel('Gaming Device')
plt.ylabel('Life Satisfaction Score')

# Device vs Social Phobia
plt.subplot(3, 1, 3)
sns.boxplot(x='Platform', y='SPIN_T', data=gamingAnxiety_df)
plt.title('Preferred Gaming Device vs Social Phobia Scores')
plt.xlabel('Gaming Device')
plt.ylabel('Social Phobia Score')

plt.tight_layout()
plt.show()

# Game genre and psychological measures
plt.figure(figsize=(15, 10))

# Genre vs Anxiety
plt.subplot(3, 1, 1)
sns.boxplot(x='Game', y='GAD_T', data=gamingAnxiety_df)
plt.title('Game Genre vs Anxiety Scores')
plt.xlabel('Game Genre')
plt.ylabel('Anxiety Score')
plt.xticks(rotation=45)

# Genre vs Life Satisfaction
plt.subplot(3, 1, 2)
sns.boxplot(x='Game', y='SWL_T', data=gamingAnxiety_df)
plt.title('Game Genre vs Life Satisfaction Scores')
plt.xlabel('Game Genre')
plt.ylabel('Life Satisfaction Score')
plt.xticks(rotation=45)

# Genre vs Social Phobia
plt.subplot(3, 1, 3)
sns.boxplot(x='Game', y='SPIN_T', data=gamingAnxiety_df)
plt.title('Game Genre vs Social Phobia Scores')
plt.xlabel('Game Genre')
plt.ylabel('Social Phobia Score')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()


## Explanation:
A boxplot is extremely useful here as it displays a range of data and highlights central tendency, and how differeng genres and platforms may affect these.

The graph shows that in mobile users tend to have GAD_T, SWL_T and SPIN_T scores compared to PC and console users. Interestingly, PC players noticeably have more outliers compared to the rest, leaning towards high GAD_T and SPIN_T.

Though there are differences in the median scores, the most striking observations about the game genre graphs are the high amount of outliers presenting high GAD_T and high SPIN_T scores for games like League of Legends and Counter Strike, games known to be highly competitive. This may present an opportunity for feature engineering, classifying the games with regards to how competitive they are.

## Conclusion:

Anxiety, Social Phobia and Satisfaction with life tends to be higher for mobile users. High-score outliers are also noticeable with regards to social phobia and anxiety for competitive games.

## RESEARCH QUESTION
After going through the Exploratory Data Analysis, our final research question is as follows:
- **Do anxiety, life satisfaction and social phobia have correlations with the game genre the person is playing?**

## Significance
We would like to see the correlation of data when it comes to anxiety, life satisfaction, and social phobia when it comes to the game genre. We would like to understand this for a couple of reasons.

1. Mental Health Awareness :  Identifying the potential impact of certain games on mental health can help people veer away from game genres that will worsen their conditions. For example, we wouldn't want an anxious person playing a game that would make them more anxious.
2. Game Design : A supporting reason is that game designers could design games that could lessen anxiety, raise life satisfaction and lessen social phobia through games. Sort of a therapeutic experience through gaming.
3. Academic Research : As the research has data and statistics backing it up, this could help in to contribute to the broader field of psychology and behavioral studies, providing insights into how modern technology and entertainment affect mental health.